# Add Information to Paul’s Oral Drug Set

### Authors: Barbara Zdrazil, Lina Heinzke
### 02/2023

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools

In [2]:
# notebook settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

In [3]:
chembl_version = '32'
base_path = '/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/'
path_results = base_path+'results/'
path_sqlite3_database = base_path+'data/chembl_'+chembl_version+'/chembl_'+chembl_version+'_sqlite/chembl_'+chembl_version+'.db'
chembl_con = sqlite3.connect(path_sqlite3_database)
oral_drugs_path = base_path+'data/2023_02_01_Oral_drug_set.xlsx'

# Define Methods to Process Dataset

## Map SMILES to ChEMBL pref_name and id

Get ChEMBL mapping from smiles to pref_name and id.

In [4]:
sql = '''
SELECT DISTINCT mh.parent_molregno, 
    md.chembl_id as parent_chembl_id, md.pref_name as parent_pref_name, 
    struct.canonical_smiles
FROM compound_properties cp
INNER JOIN molecule_hierarchy mh
    ON cp.molregno = mh.parent_molregno
INNER JOIN compound_structures struct
    ON mh.parent_molregno = struct.molregno
INNER JOIN molecule_dictionary md
    ON mh.parent_molregno = md.molregno   -- compound information based on parent compound
'''

df_cpd_struct = pd.read_sql_query(sql, con=chembl_con)
df_cpd_struct.head()

,parent_molregno,parent_chembl_id,parent_pref_name,canonical_smiles
0,517180,CHEMBL1,None,COc1ccc2c(c1)OC[C@H]1[C@@H]2C2=C(OC1(C)C)C1=C(C(=O)C2=O)[C@H]2c3ccc(OC)cc3OC[C@H]2C(C)(C)O1
1,250,CHEMBL10,SB-203580,C[S+]([O-])c1ccc(-c2nc(-c3ccc(F)cc3)c(-c3ccncc3)[nH]2)cc1
2,12356,CHEMBL100,LEVCROMAKALIM,CC1(C)Oc2ccc(C#N)cc2[C@@H](N2CCCC2=O)[C@@H]1O
3,111185,CHEMBL1000,CETIRIZINE,O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
4,7079,CHEMBL10000,None,O=c1oc(Nc2ccc(I)cc2)nc2ccccc12


Method to map SMILES to canonical SMILES (RDKit-based). Returns canonical smiles and bool whether the calculation was successful. 

In [5]:
def to_canonical(smiles):
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smiles)), False
    except:
        return smiles, True

Map ChEMBL canonical_smiles to RDKit-based canonical SMILES.

In [6]:
df_cpd_struct[["rdkit_canonical_smiles", "rdkit_smiles_problem"]] = df_cpd_struct.apply(lambda row: to_canonical(row.canonical_smiles), 
                                                                           axis='columns', result_type='expand')

[21:46:44] WARNING: not removing hydrogen atom without neighbors
[21:47:45] WARNING: not removing hydrogen atom without neighbors
[21:47:45] WARNING: not removing hydrogen atom without neighbors
[21:47:45] WARNING: not removing hydrogen atom without neighbors
[21:47:50] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 10 11 15 16 17 19 20 21

[21:48:08] WARNING: not removing hydrogen atom without neighbors
[21:48:08] WARNING: not removing hydrogen atom without neighbors
[21:48:08] WARNING: not removing hydrogen atom without neighbors
[21:48:08] WARNING: not removing hydrogen atom without neighbors
[21:49:04] WARNING: not removing hydrogen atom without neighbors
[21:49:04] WARNING: not removing hydrogen atom without neighbors


Method to calculate mapping from given SMILES to ChEMBL ID (molregno + ChEMBL ID) and pref_name. A SMILES might map to more than one ID.

In [7]:
def to_chembl_id(oral_drug_smiles):
    chembl_smiles = set(df_cpd_struct["rdkit_canonical_smiles"])
    if oral_drug_smiles in chembl_smiles:
        chembl_molregno = list(df_cpd_struct[df_cpd_struct["rdkit_canonical_smiles"] == oral_drug_smiles]['parent_molregno'])
        chembl_id = list(df_cpd_struct[df_cpd_struct["rdkit_canonical_smiles"] == oral_drug_smiles]['parent_chembl_id'])
        chembl_name = list(df_cpd_struct[df_cpd_struct["rdkit_canonical_smiles"] == oral_drug_smiles]['parent_pref_name'])
#       # output if there is more than one compound in ChEMBL for a SMILES
        if len(chembl_molregno) > 1:
            print(oral_drug_smiles, chembl_molregno, chembl_id, chembl_name)
        return chembl_molregno, chembl_id, chembl_name 
    else:
        return None, None, None

## Map Compounds to Targets in the drug_mechanism Table

Get interacting compound-target pairs from the drug_mechanism table.

In [8]:
sql = '''
SELECT DISTINCT mh.parent_molregno, dm.tid, md.chembl_id, td.pref_name
FROM drug_mechanism dm
INNER JOIN molecule_hierarchy mh
    ON dm.molregno = mh.molregno
INNER JOIN molecule_dictionary md
    ON mh.parent_molregno = md.molregno
INNER JOIN target_dictionary td
    ON dm.tid = td.tid
WHERE dm.disease_efficacy = 1
    and dm.tid is not null
'''

df_dti = pd.read_sql_query(sql, con=chembl_con)
df_dti

,parent_molregno,tid,chembl_id,pref_name
0,1124,11060,CHEMBL19,Carbonic anhydrase VII
1,675068,10193,CHEMBL1201117,Carbonic anhydrase I
2,1125,10193,CHEMBL20,Carbonic anhydrase I
3,1085,10193,CHEMBL17,Carbonic anhydrase I
4,1124,10193,CHEMBL19,Carbonic anhydrase I
...,...,...,...,...
6258,1407411,112,CHEMBL2135460,Vasopressin V2 receptor
6259,51961,120553,CHEMBL37161,Tubulin beta chain
6260,51961,120554,CHEMBL37161,Tubulin beta chain
6261,442342,22228,CHEMBL272427,Molecular identity unknown


Method to calculate mapping from compound ChEMBL id to targets with which they are known to interact based on the drug_mechanism table.  
Returns target pref_names and ids and bool whether the compound is in the drug_mechanism table. There can be multiple target pref_names / ids per compound.

In [9]:
def in_dm_table(oral_drug_id):
    if oral_drug_id in set(df_dti['parent_molregno']):
        target_names = list(df_dti[df_dti["parent_molregno"] == oral_drug_id]['pref_name'])
        tids = list(df_dti[df_dti["parent_molregno"] == oral_drug_id]['tid'])
        target_chembl_ids = list(df_dti[df_dti["parent_molregno"] == oral_drug_id]['chembl_id'])
        return True, target_names, tids, target_chembl_ids
    else:
        return False, None, None, None

## Methods to Add Target Class Annotations Based on ChEMBL Data

Add information about level 1 and level 2 target class annotations in ChEMBL.

In [10]:
sql = '''
SELECT DISTINCT tc.tid, 
    pc.protein_class_id, pc.pref_name, pc.short_name, pc.protein_class_desc, pc.definition
FROM protein_classification pc
-- join several tables to get the corresponding target id
INNER JOIN component_class cc
    ON pc.protein_class_id = cc.protein_class_id
INNER JOIN component_sequences cs
    ON cc.component_id = cs.component_id
INNER JOIN target_components tc
    ON cs.component_id = tc.component_id
'''

df_target_classes = pd.read_sql_query(sql, con=chembl_con)
df_target_classes

,tid,protein_class_id,pref_name,short_name,protein_class_desc,definition
0,1,646,Hydrolase,Hydrolase,enzyme hydrolase,A group of enzymes that catalyze the hydrolysis of a chemical bond
1,2,1133,ABCC subfamily,MRP,transporter ntpase atp binding cassette mrp,A sequence-related subfamily of ATP-BINDING CASSETTE TRANSPORTERS that actively transport organi...
2,3,104,Phosphodiesterase 5A,PDE_5A,enzyme phosphodiesterase pde_5 pde_5a,None
3,4,1583,Voltage-gated calcium channel,VG CA,ion channel vgc vg ca,Voltage-dependent cell membrane glycoproteins selectively permeable to calcium ions. They are ca...
4,5,422,Nicotinic acetylcholine receptor alpha subunit,CHRN alpha,ion channel lgic ach chrn alpha,None
...,...,...,...,...,...,...
11700,120595,601,Unclassified protein,Unclassified,unclassified,None
11701,120596,601,Unclassified protein,Unclassified,unclassified,None
11702,120597,601,Unclassified protein,Unclassified,unclassified,None
11703,120598,601,Unclassified protein,Unclassified,unclassified,None


Query the protein_classification table for the protein classification hierarchy and merge it with the target class information for specific tids.

In [11]:
sql = '''
WITH RECURSIVE pc_hierarchy AS (
    SELECT protein_class_id,
            parent_id,
            class_level,
            pref_name AS names
    FROM protein_classification
    WHERE parent_id IS NULL

    UNION ALL
   
    SELECT pc.protein_class_id,
        pc.parent_id,
        pc.class_level,
        -- recursively add current protein classification pref_name to string, separated by |
        pc_hierarchy.names || '|' || pc.pref_name 
    FROM protein_classification pc, pc_hierarchy
    WHERE pc.parent_id = pc_hierarchy.protein_class_id
)
SELECT *
FROM pc_hierarchy
'''


target_class_hierarchy = pd.read_sql_query(sql, con=chembl_con)
target_class_hierarchy[['l0', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6']] = target_class_hierarchy['names'].str.split('|', expand=True)
target_class_hierarchy = target_class_hierarchy[target_class_hierarchy['protein_class_id'] != 0][['protein_class_id', 'l1', 'l2']]
df_target_classes = df_target_classes.merge(target_class_hierarchy, on = 'protein_class_id', how = 'left')
df_target_classes

,tid,protein_class_id,pref_name,short_name,protein_class_desc,definition,l1,l2
0,1,646,Hydrolase,Hydrolase,enzyme hydrolase,A group of enzymes that catalyze the hydrolysis of a chemical bond,Enzyme,Hydrolase
1,2,1133,ABCC subfamily,MRP,transporter ntpase atp binding cassette mrp,A sequence-related subfamily of ATP-BINDING CASSETTE TRANSPORTERS that actively transport organi...,Transporter,Primary active transporter
2,3,104,Phosphodiesterase 5A,PDE_5A,enzyme phosphodiesterase pde_5 pde_5a,None,Enzyme,Phosphodiesterase
3,4,1583,Voltage-gated calcium channel,VG CA,ion channel vgc vg ca,Voltage-dependent cell membrane glycoproteins selectively permeable to calcium ions. They are ca...,Ion channel,Voltage-gated ion channel
4,5,422,Nicotinic acetylcholine receptor alpha subunit,CHRN alpha,ion channel lgic ach chrn alpha,None,Ion channel,Ligand-gated ion channel
...,...,...,...,...,...,...,...,...
11700,120595,601,Unclassified protein,Unclassified,unclassified,None,Unclassified protein,None
11701,120596,601,Unclassified protein,Unclassified,unclassified,None,Unclassified protein,None
11702,120597,601,Unclassified protein,Unclassified,unclassified,None,Unclassified protein,None
11703,120598,601,Unclassified protein,Unclassified,unclassified,None,Unclassified protein,None


Summarise the information for a target id with several assigned target classes of level 1 into one description. If a target id has more than one assigned target class, the target class 'Unclassified protein' is discarded.

In [12]:
level = 'l1'
between_str_join = '|'
target_classes_level1 = df_target_classes[['tid', level]].drop_duplicates().dropna()

# remove 'Unclassified protein' from targets with more than one target class, level 1
nof_classes = target_classes_level1.groupby(['tid'])[level].count()
target_classes_level1 = target_classes_level1[
    (target_classes_level1['tid'].isin(nof_classes[nof_classes == 1].index.tolist())) 
    | ((target_classes_level1['tid'].isin(nof_classes[nof_classes > 1].index.tolist())) 
       & (target_classes_level1['l1'] != 'Unclassified protein'))]

target_classes_level1['target_class_l1'] = target_classes_level1.groupby(['tid'])[level].transform(lambda x: between_str_join.join(sorted(x)))
target_classes_level1 = target_classes_level1[['tid', 'target_class_l1']].drop_duplicates()

Repeat the summary step for target classes of level 2.

In [13]:
level = 'l2'
target_classes_level2 = df_target_classes[['tid', level]].drop_duplicates().dropna()
target_classes_level2['target_class_l2'] = target_classes_level2.groupby(['tid'])[level].transform(lambda x: between_str_join.join(sorted(x)))
target_classes_level2 = target_classes_level2[['tid', 'target_class_l2']].drop_duplicates()

## Methods to Add RDKit-Based Compound Descriptors

## Built-in Compound Descriptors

Add relevant compound descriptors using built-in RDKit methods. 

In [14]:
def add_RDKit_columns(oral_drugs):
    # add a column with RDKit molecules, used to calculate the descriptors
    PandasTools.AddMoleculeColumnToFrame(oral_drugs, smiles_column, 'mol', includeFingerprints=False)

    oral_drugs.loc[:,'fraction_csp3'] = oral_drugs['mol'].apply(Descriptors.FractionCSP3)
    oral_drugs.loc[:,'num_aliphatic_carbocycles'] = oral_drugs['mol'].apply(Descriptors.NumAliphaticCarbocycles)
    oral_drugs.loc[:,'num_aliphatic_heterocycles'] = oral_drugs['mol'].apply(Descriptors.NumAliphaticHeterocycles)
    oral_drugs.loc[:,'num_aliphatic_rings'] = oral_drugs['mol'].apply(Descriptors.NumAliphaticRings)
    oral_drugs.loc[:,'num_aromatic_carbocycles'] = oral_drugs['mol'].apply(Descriptors.NumAromaticCarbocycles)
    oral_drugs.loc[:,'num_aromatic_heterocycles'] = oral_drugs['mol'].apply(Descriptors.NumAromaticHeterocycles)
    oral_drugs.loc[:,'num_aromatic_rings'] = oral_drugs['mol'].apply(Descriptors.NumAromaticRings)
    oral_drugs.loc[:,'num_heteroatoms'] = oral_drugs['mol'].apply(Descriptors.NumHeteroatoms)
    oral_drugs.loc[:,'num_saturated_carbocycles'] = oral_drugs['mol'].apply(Descriptors.NumSaturatedCarbocycles)
    oral_drugs.loc[:,'num_saturated_heterocycles'] = oral_drugs['mol'].apply(Descriptors.NumSaturatedHeterocycles)
    oral_drugs.loc[:,'num_saturated_rings'] = oral_drugs['mol'].apply(Descriptors.NumSaturatedRings)
    oral_drugs.loc[:,'ring_count'] = oral_drugs['mol'].apply(Descriptors.RingCount)
    oral_drugs.loc[:,'num_stereocentres'] = oral_drugs['mol'].apply(Chem.rdMolDescriptors.CalcNumAtomStereoCenters)

    # add scaffolds
    PandasTools.AddMurckoToFrame(oral_drugs, 'mol', 'scaffold_w_stereo')
    # remove stereo information of the molecule to add scaffolds without stereo information
    oral_drugs['mol'].apply(Chem.RemoveStereochemistry)
    PandasTools.AddMurckoToFrame(oral_drugs, 'mol', 'scaffold_wo_stereo')

    # drop the column with RDKit molecules
    return oral_drugs.drop(['mol'] , axis=1)

## Aromaticity Descriptors

Add descriptors for aromaticity, using an RDKit-based method.

In [15]:
def calculate_aromatic_atoms(smiles_set):
    aromatic_atoms_dict = dict()
    aromatic_c_dict = dict()
    aromatic_n_dict = dict()
    aromatic_hetero_dict = dict()
    
    for smiles in tqdm(smiles_set):
        mol = Chem.MolFromSmiles(smiles)
        aromatic_atoms_dict[smiles] = sum(mol.GetAtomWithIdx(i).GetIsAromatic() for i in range(mol.GetNumAtoms()))
        aromatic_c_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() == 6)) for i in range(mol.GetNumAtoms()))
        aromatic_n_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() == 7)) for i in range(mol.GetNumAtoms()))
        aromatic_hetero_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() != 6) & (mol.GetAtomWithIdx(i).GetAtomicNum() != 1)) for i in range(mol.GetNumAtoms()))
        
    return aromatic_atoms_dict, aromatic_c_dict, aromatic_n_dict, aromatic_hetero_dict

In [16]:
def add_aromaticity_columns(oral_drugs):
    # use df_combined_w_smiles to exclude null values
    smiles_set = set(oral_drugs[smiles_column])
    aromatic_atoms_dict, aromatic_c_dict, aromatic_n_dict, aromatic_hetero_dict = calculate_aromatic_atoms(smiles_set)

    oral_drugs['aromatic_atoms'] = oral_drugs[smiles_column].map(aromatic_atoms_dict)
    oral_drugs['aromatic_c'] = oral_drugs[smiles_column].map(aromatic_c_dict)
    oral_drugs['aromatic_n'] = oral_drugs[smiles_column].map(aromatic_n_dict)
    oral_drugs['aromatic_hetero'] = oral_drugs[smiles_column].map(aromatic_hetero_dict)
    return oral_drugs

# Process Dataset

In [17]:
sheet_names = ['All', 'Additions info', 'Post90 with targets annotated']
smiles_columns = ['SMILES', 'Smiles', 'SMILES']
existing_columns = [11, 9, 8]
results = []

Method to calculate all annotations for a sheet in the excel file.

In [18]:
def get_result(sheet_name, smiles_column):
    # read file
    print('read file')
    oral_drugs = pd.read_excel(oral_drugs_path, sheet_name=sheet_name)
    
    # map smiles to canonical smiles
    print('map smiles to canonical smiles')
    oral_drugs[["rdkit_canonical_smiles", 
                "rdkit_smiles_problem"]] = oral_drugs.apply(lambda row: to_canonical(row[smiles_column]), 
                                                            axis='columns', result_type='expand')
    # map compound to chembl_id and pref_name
    print('map compound to chembl_id and pref_name')
    oral_drugs[["chembl_parent_molregno", 
                "chembl_parent_id",
                "chembl_parent_pref_name"]] = oral_drugs.apply(lambda row: to_chembl_id(row.rdkit_canonical_smiles), 
                                                        axis='columns', result_type='expand')
    # SMILES may map to more than one 
    oral_drugs = oral_drugs.explode(['chembl_parent_molregno', 'chembl_parent_id', 'chembl_parent_pref_name'])
    oral_drugs = oral_drugs.astype({'chembl_parent_molregno': 'Int64'})
    
    # map to targets the compound is known to interact with based on the drug_mechanism table
    print('map to targets the compound is known to interact with based on the drug_mechanism table')
    oral_drugs[["in_dm_table", 
            "chembl_target_name", 
            "chembl_tid", 
            "chembl_target_id"]] = oral_drugs.apply(lambda row: in_dm_table(row.chembl_parent_molregno), 
                                                      axis='columns', result_type='expand')
    # compound may appear more than once in drug_mechanism table
    oral_drugs = oral_drugs.explode(['chembl_target_name', 'chembl_tid', 'chembl_target_id'])
    
    # add target class annotations based on target ids
    print('add target class annotations based on target ids')
    oral_drugs = oral_drugs.merge(target_classes_level1[['tid', 'target_class_l1']], left_on='chembl_tid', right_on='tid', how = 'left').drop(columns=['tid'])
    oral_drugs = oral_drugs.merge(target_classes_level2, left_on='chembl_tid', right_on='tid', how = 'left').drop(columns=['tid'])
    
    # add RDKit-based properties
    print('add RDKit-based properties')
    oral_drugs = add_RDKit_columns(oral_drugs)
    oral_drugs = add_aromaticity_columns(oral_drugs)
    return oral_drugs

Calculate the results for all three excel sheets.

In [19]:
for sheet_name, smiles_column in zip(sheet_names, smiles_columns):
    oral_drugs = pd.read_excel(oral_drugs_path, sheet_name=sheet_name)
    print(sheet_name)
    print('{:10} {:4}'.format('#Rows:', len(oral_drugs)))
    print('{:10} {:4}'.format('#SMILES:', len(set(oral_drugs[smiles_column]))))
    print('{:10} {:4}'.format('#Rows w/o SMILES:', len(oral_drugs[oral_drugs[smiles_column].isnull()])))
    results.append(get_result(sheet_name, smiles_column))
    print('----------------')

All
#Rows:     2059
#SMILES:   2057
#Rows w/o SMILES:    0
read file
map smiles to canonical smiles
map compound to chembl_id and pref_name
CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(OC)[C@@H](O)[C@H](C)O2)[C@H](C)[C@@H](O[C@@H]2O[C@H](C)C[C@H](N(C)C)[C@H]2O)[C@](C)(O)C[C@@H](C)[C@@H]2N[C@@H](COCCOC)O[C@H]([C@H]2C)[C@]1(C)O [699423, 1592294] ['CHEMBL1237072', 'CHEMBL3039471'] ['DIRITHROMYCIN', 'DIRITHROMYCIN']
Cc1oc(=O)oc1COC(=O)[C@@H]1N2C(=O)[C@@H](NC(=O)[C@H](N)c3ccccc3)[C@H]2SC1(C)C [1378599, 1946697] ['CHEMBL2106329', 'CHEMBL3580454'] ['LENAMPICILLIN', 'LENAMPICILLIN']
C=CC[C@@H]1/C=C(\C)C[C@H](C)C[C@H](OC)[C@H]2O[C@@](O)(C(=O)C(=O)N3CCCC[C@H]3C(=O)O[C@H](/C(C)=C/[C@@H]3CC[C@@H](O)[C@H](OC)C3)[C@H](C)[C@@H](O)CC1=O)[C@H](C)C[C@@H]2OC [924, 2435351] ['CHEMBL269732', 'CHEMBL4564923'] ['TACROLIMUS', None]
map to targets the compound is known to interact with based on the drug_mechanism table
add target class annotations based on target ids
add RDKit-based properties


100%|██████████████████████████████████████████████████████████████████████| 2057/2057 [00:00<00:00, 5898.44it/s]


----------------
Additions info
#Rows:       25
#SMILES:     25
#Rows w/o SMILES:    0
read file
map smiles to canonical smiles
map compound to chembl_id and pref_name
map to targets the compound is known to interact with based on the drug_mechanism table
add target class annotations based on target ids
add RDKit-based properties


100%|██████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 4268.40it/s]

----------------
Post90 with targets annotated
#Rows:      437
#SMILES:    437
#Rows w/o SMILES:    0
read file
map smiles to canonical smiles


map compound to chembl_id and pref_name
map to targets the compound is known to interact with based on the drug_mechanism table
add target class annotations based on target ids
add RDKit-based properties


100%|████████████████████████████████████████████████████████████████████████| 437/437 [00:00<00:00, 4483.48it/s]

----------------


## Postprocessing (Types, Rounding)

In [20]:
for i in range(len(results)):
    results[i] = results[i].where(pd.notnull(results[i]), None)
    results[i] = results[i].astype({
        'in_dm_table': 'bool', 
        'rdkit_smiles_problem': 'bool', 
        'chembl_tid': 'Int64', 
        'num_aliphatic_carbocycles': 'Int64',
        'num_aliphatic_heterocycles': 'Int64',
        'num_aliphatic_rings': 'Int64',
        'num_aromatic_carbocycles': 'Int64',
        'num_aromatic_heterocycles': 'Int64',
        'num_aromatic_rings': 'Int64',
        'num_heteroatoms': 'Int64',
        'num_saturated_carbocycles': 'Int64',
        'num_saturated_heterocycles': 'Int64',
        'num_saturated_rings': 'Int64',
        'ring_count': 'Int64',
        'num_stereocentres': 'Int64',
        'aromatic_atoms': 'Int64',
        'aromatic_c': 'Int64',
        'aromatic_n': 'Int64',
        'aromatic_hetero': 'Int64'
    })

In [21]:
# Round float columns to 4 decimal places
# for result, existing_cols in zip(results, existing_columns):
for i in range(len(results)):
    existing_cols = existing_columns[i]
    decimal_places = 4
    for col_id, (col, dtype) in enumerate(results[i].dtypes.to_dict().items()):
        if col_id > existing_cols:
            if ((dtype == 'float64') or (dtype == 'Float64')):
                results[i][col] = results[i][col].round(decimals=decimal_places)

## Sanity Checks

Check that columns have expected types (by hand).

In [22]:
for result, existing_cols in zip(results, existing_columns):
    print("{:3} {:50} {:10} {}".format("", "column", "type", "#null values"))
    for i, (col, dtype) in enumerate(result.dtypes.to_dict().items()):
        if i > existing_cols:
            print("{:3} {:50} {:10} {}".format(i, col, str(dtype), len(result[result[col].isnull()])))
    print()

    column                                             type       #null values
 12 rdkit_canonical_smiles                             object     0
 13 rdkit_smiles_problem                               bool       0
 14 chembl_parent_molregno                             Int64      125
 15 chembl_parent_id                                   object     125
 16 chembl_parent_pref_name                            object     185
 17 in_dm_table                                        bool       0
 18 chembl_target_name                                 object     846
 19 chembl_tid                                         Int64      846
 20 chembl_target_id                                   object     846
 21 target_class_l1                                    object     904
 22 target_class_l2                                    object     979
 23 fraction_csp3                                      float64    0
 24 num_aliphatic_carbocycles                          Int64      0
 25 num_aliphatic_het

Check if there are mixed types in columns with dtype=object.

In [23]:
# check that there are no mixed types in object columns
for result, existing_cols, sheet_name in zip(results, existing_columns, sheet_names):
    print(sheet_name)
    issue_ctr = 0
    for i, (col, dtype) in enumerate(result.dtypes.to_dict().items()):
        if i > existing_cols:
            if dtype == object:
                a = set(result[col])
                b = set(result[col].astype(str))
                x = a-b
                y = b-a
                # is there a difference in the two sets
                if(len(x) > 0 or len(y) > 0):
                    if not ((len(x.difference({None})) == 0 and len(y.difference({'None'})) == 0)) and \
                        not ((len(x.difference({np.nan})) == 0 and len(y.difference({'nan'})) == 0)):
                        print("Mixed types in column ", col)
                        print(a-b, '/', b-a)
                        issue_ctr += 1

    print("#Problems:", issue_ctr, '\n')

All
#Problems: 0 

Additions info
#Problems: 0 

Post90 with targets annotated
#Problems: 0 



Check if any columns contain nan or null which aren't recognised as null values. 

In [24]:
# Do any columns have potential issues with null types?
for result, existing_cols, sheet_name in zip(results, existing_columns, sheet_names):
    print(sheet_name)
    issue_ctr = 0
    for i, (col, dtype) in enumerate(result.dtypes.to_dict().items()):
        if 'nan' in set(result[result[col].notnull()][col].astype(str)):
            print("Issue with nan in column", col)
            issue_ctr += 1
        if 'null' in set(result[result[col].notnull()][col].astype(str)):
            print("Issue with null in column", col)
            issue_ctr += 1

    print("#Problems:", issue_ctr, '\n')

All
#Problems: 0 

Additions info
#Problems: 0 

Post90 with targets annotated
#Problems: 0 



## Writing Results

Write full dataset to file.

In [25]:
# Write results to new sheet
dataset_all_name = path_results+'Oral_drugs_chembl.xlsx'
with pd.ExcelWriter(dataset_all_name,engine='xlsxwriter') as writer: 
    for result, sheet_name, num_cols in zip(results, sheet_names, existing_columns):
        columns = list(result.columns[:num_cols+1]) + \
            ['rdkit_canonical_smiles', 
             'chembl_parent_molregno', 'chembl_parent_id', 'chembl_parent_pref_name',
             'in_dm_table', 'chembl_target_name', 'chembl_tid', 'chembl_target_id',
             'target_class_l1', 'target_class_l2', 
             'fraction_csp3',
             'num_aliphatic_carbocycles', 'num_aliphatic_heterocycles', 'num_aliphatic_rings', 
             'num_aromatic_carbocycles', 'num_aromatic_heterocycles', 'num_aromatic_rings', 
             'num_heteroatoms', 'num_saturated_carbocycles', 'num_saturated_heterocycles',
             'num_saturated_rings', 'ring_count', 'num_stereocentres',
             'scaffold_w_stereo', 'scaffold_wo_stereo', 
             'aromatic_atoms', 'aromatic_c', 'aromatic_n', 'aromatic_hetero']
        result[columns].to_excel(writer, sheet_name=sheet_name, index = False)

Write Dataset restricted to RDKit-based properties to file.

In [26]:
# Write restricted results to new sheet
dataset_restricted_name = path_results+'Oral_drugs_chembl_restricted.xlsx'
with pd.ExcelWriter(dataset_restricted_name) as writer: 
    for result, sheet_name, num_cols in zip(results, sheet_names, existing_columns):
        columns = list(result.columns[:num_cols+1]) + \
            ['fraction_csp3',
             'num_aliphatic_carbocycles', 'num_aliphatic_heterocycles', 'num_aliphatic_rings', 
             'num_aromatic_carbocycles', 'num_aromatic_heterocycles', 'num_aromatic_rings', 
             'num_heteroatoms', 'num_saturated_carbocycles', 'num_saturated_heterocycles',
             'num_saturated_rings', 'ring_count', 'num_stereocentres',
             'scaffold_w_stereo', 'scaffold_wo_stereo', 
             'aromatic_atoms', 'aromatic_c', 'aromatic_n', 'aromatic_hetero']
        result[columns].drop_duplicates().to_excel(writer, sheet_name=sheet_name, index = False)

# Sanity Checks

Check that output files can be written and read and are identical to the results.

In [27]:
for read_filename in [dataset_all_name, dataset_restricted_name]:
    print(read_filename)
    for result, sheet_name in zip(results, sheet_names):
        result_copy = result.copy().drop(columns=['rdkit_smiles_problem'])
        result_copy = result_copy.replace('', None).reset_index(drop=True)

        read_file = pd.read_excel(read_filename, sheet_name=sheet_name)
        read_file = read_file.where(pd.notnull(read_file), None)
        read_file = read_file.replace('', None).reset_index(drop=True)

        if read_filename == dataset_all_name:
            read_file = read_file.astype({
                'chembl_parent_molregno': 'Int64', 
                'in_dm_table': 'bool', 
                'chembl_tid': 'Int64', 
            })
        else: #read_filename == dataset_restricted_name
            result_copy = result_copy[read_file.columns].drop_duplicates().reset_index(drop=True)
        
        read_file = read_file.astype({
                'num_aliphatic_carbocycles': 'Int64',
                'num_aliphatic_heterocycles': 'Int64',
                'num_aliphatic_rings': 'Int64',
                'num_aromatic_carbocycles': 'Int64',
                'num_aromatic_heterocycles': 'Int64',
                'num_aromatic_rings': 'Int64',
                'num_heteroatoms': 'Int64',
                'num_saturated_carbocycles': 'Int64',
                'num_saturated_heterocycles': 'Int64',
                'num_saturated_rings': 'Int64',
                'ring_count': 'Int64',
                'num_stereocentres': 'Int64',
                'aromatic_atoms': 'Int64',
                'aromatic_c': 'Int64',
                'aromatic_n': 'Int64',
                'aromatic_hetero': 'Int64'
        })

        print("{:40} file is ok: {}".format(sheet_name, read_file.equals(result_copy)))
    print("----------")

/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/results/Oral_drugs_chembl.xlsx
All                                      file is ok: True
Additions info                           file is ok: True
Post90 with targets annotated            file is ok: True
----------
/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/results/Oral_drugs_chembl_restricted.xlsx
All                                      file is ok: True
Additions info                           file is ok: True
Post90 with targets annotated            file is ok: True
----------


Check that output files can be written and read and are identical to reading the original dataset.

In [28]:
for read_filename in [dataset_all_name, dataset_restricted_name]:
    print(read_filename)
    for result, sheet_name in zip(results, sheet_names):
        original_file = pd.read_excel(oral_drugs_path, sheet_name=sheet_name)

        read_file = pd.read_excel(read_filename, sheet_name=sheet_name)
        read_file = read_file[original_file.columns].drop_duplicates().reset_index(drop=True)

        print("{:40} file is ok: {}".format(sheet_name, read_file.equals(original_file)))
    print("----------")

/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/results/Oral_drugs_chembl.xlsx
All                                      file is ok: True
Additions info                           file is ok: True
Post90 with targets annotated            file is ok: True
----------
/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/results/Oral_drugs_chembl_restricted.xlsx
All                                      file is ok: True
Additions info                           file is ok: True
Post90 with targets annotated            file is ok: True
----------
